In [30]:
# Import packages
import os
import glob
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats as stats
import datetime

# Import dabest for plotting. Set path to DABEST folder.
import sys
sys.path.insert(0, '/Users/CeceliaAndrews/DABEST-python')
import dabest

# Import Cece's migration analysis and bootstrapping modules.
# Change path as needed
sys.path.insert(0, '/Users/CeceliaAndrews/mike')
import mig_analysis as ma
import cece_bootstrapping as cbs

In [31]:
# Set path to data we want to analyze
path = os.path.abspath('')+'/CSVs/'

# Load data as a dataframe with the columns we want.
# get_ultimate_dataframe includes tests to confirm the csv files look how we want them.
# The function will print info about the csvs.
df_data = ma.get_ultimate_dataframe(path)
df_data.head()

Number of files in this folder: 50


In first file:
Number of rows: 2
Number of columns: 6
Column names: [' ' 'Label' 'Area' 'Mean' 'IntDen' 'RawIntDen']
Image name: 20180715_SMPD3MONC11m3bCat_0,8mM2ugul_Pax7GFP_Emb6_8ss_10x
Image name components:
20180715
SMPD3MONC11m3bCat
0,8mM2ugul
Pax7GFP
Emb6
8ss
10x


You're good to go! All files match expected format.


,Date,Treatment,Dose,Image,Somites,Control Area,Experiment Area,Exp/Ctl Area,Norm Control Area,Norm Experiment Area
0,20180715,SMPD3MONC11m3bCat,"0,8mM2ugul","20180715_SMPD3MONC11m3bCat_0,8mM2ugul_Pax7GFP_...",8ss,72103.705,23918.319,0.331721,1.119337,0.371308
1,20180630,SMPD3MO,"0,8mM","20180630_SMPD3MO_0,8mM_Pax7_Emb5_8ss_10x",8ss,80452.846,47922.588,0.595661,0.955197,0.568973
2,20181119,SMPD3MO,"0,6mM","20181119_SMPD3MO_0,6mM_Pax7_Emb6_7ss_10x",7ss,47346.082,27928.306,0.589876,0.562129,0.331586
3,20180630,SMPD3MO,"0,8mM","20180630_SMPD3MO_0,8mM_Pax7_Emb3_9ss_10x",9ss,146043.286,47621.453,0.326078,1.733937,0.565398
4,20180514,SMPD3MO,"0,8mM","20180514_SMPD3MO_0,8mM_Pax7Laminin_Emb6_8ss_10x",8ss,72508.322,52025.129,0.717506,0.860874,0.617682


In [3]:
# Save dataframe as csv
df_data.to_csv(ma.get_analysis_date() + '_MigrationResults.csv')

In [4]:
# Print summary statistics for each experiment
ma.print_stats(df_data)

Mean
                   Control Area  Experiment Area  Exp/Ctl Area
Treatment                                                     
SMPD3MO            84226.407037     49172.997333      0.603374
SMPD3MONC11m3bCat  64416.428750     28288.802625      0.462259
SMPD3gRNA1DNA      84971.282750     63640.824000      0.753154
SMPD3gRNA1RNP      40021.131571     32142.082286      0.804300


SEM
                   Control Area  Experiment Area  Exp/Ctl Area
Treatment                                                     
SMPD3MO             6747.873789      4089.042977      0.028058
SMPD3MONC11m3bCat   4855.633971      2025.209200      0.054928
SMPD3gRNA1DNA       7102.405584      6431.628777      0.061069
SMPD3gRNA1RNP       4141.117489      3569.071976      0.034698


Correlation
                                   Control Area  Experiment Area
Treatment                                                       
SMPD3MO           Control Area         1.000000         0.804412
                  Experi

In [23]:
# Create dataframe of summary statistics for each experiment
df_stats = ma.get_stats_df(df_data)
df_stats

,Treatment,Control Area Mean,Control Area SEM,Experiment Area Mean,Experiment Area SEM,Exp/Ctl Mean,Exp/Ctl SEM,Correlation
0,SMPD3MONC11m3bCat,64416.428750,4855.633971,28288.802625,2025.209200,0.462259,0.054928,-0.209264
1,SMPD3MO,84226.407037,6747.873789,49172.997333,4089.042977,0.603374,0.028058,0.804412
2,SMPD3gRNA1RNP,40021.131571,4141.117489,32142.082286,3569.071976,0.804300,0.034698,0.929832
3,SMPD3gRNA1DNA,84971.282750,7102.405584,63640.824000,6431.628777,0.753154,0.061069,0.638671


In [29]:
# Let's do some bootstrapping and add the confidence interval and p value to stats dataframe

# Get list of individual treatments
treatment_list = df_stats['Treatment']

# Set up lists to store stats
conf_int = []
bca_low = []
bca_high = []
pvalue = []

for treatment in treatment_list:
    # Extract data for each treatment
    df_treat = df_data.loc[df_data['Treatment'] == treatment]
    df_treat = df_treat.reset_index()
    
    # Get normalized control and experiment areas
    x1 = df_treat['Norm Control Area']
    x2 = df_treat['Norm Experiment Area']
    
    # Conduct paired means bootstrapping
    ci, low, high, p = cbs.bootstrap(x1, x2=x2, paired=True)
    
    # Add stats to list
    conf_int.append(ci)
    bca_low.append(low)
    bca_high.append(high)
    pvalue.append(p)

# Add stats to stats dataframe
df_stats['Confidence Interval'] = conf_int
df_stats['BCA Low'] = bca_low
df_stats['BCA High'] = bca_high
df_stats['P Value 2 Sample Paired T Test'] = pvalue
df_stats

/Users/CeceliaAndrews/anaconda3/lib/python3.7/site-packages/scipy/stats/morestats.py:2778: UserWarning: Warning: sample size too small for normal approximation.
  warnings.warn("Warning: sample size too small for normal approximation.")
/Users/CeceliaAndrews/anaconda3/lib/python3.7/site-packages/scipy/stats/morestats.py:2778: UserWarning: Warning: sample size too small for normal approximation.
  warnings.warn("Warning: sample size too small for normal approximation.")
/Users/CeceliaAndrews/anaconda3/lib/python3.7/site-packages/scipy/stats/morestats.py:2778: UserWarning: Warning: sample size too small for normal approximation.
  warnings.warn("Warning: sample size too small for normal approximation.")


,Treatment,Control Area Mean,Control Area SEM,Experiment Area Mean,Experiment Area SEM,Exp/Ctl Mean,Exp/Ctl SEM,Correlation,Confidence Interval,BCA Low,BCA High,P Value 2 Sample Paired T Test
0,SMPD3MONC11m3bCat,64416.428750,4855.633971,28288.802625,2025.209200,0.462259,0.054928,-0.209264,95.0,-0.714522,-0.391714,3.647945e-04
1,SMPD3MO,84226.407037,6747.873789,49172.997333,4089.042977,0.603374,0.028058,0.804412,95.0,-0.525517,-0.329966,8.934936e-09
2,SMPD3gRNA1RNP,40021.131571,4141.117489,32142.082286,3569.071976,0.804300,0.034698,0.929832,95.0,-0.276209,-0.133955,2.255668e-03
3,SMPD3gRNA1DNA,84971.282750,7102.405584,63640.824000,6431.628777,0.753154,0.061069,0.638671,95.0,-0.358682,-0.102466,7.782316e-03
